# Data Science Capstone Final Assignment

# The Mover's Guide: Toronto to New York

Being the two most populous cities of their countries, Toronto and New York are metropolitan cities with plenty of migration. 
The Mover's Guide is a script which can used to find out which neighborhoods in either city is suitable for a user who wishes to move from Toronto to New York or vice versa.

It uses data from Foursquare to identify the make-up of different neighborhoods in Toronto and New York. By collecting all the data in a single dataframe, the neighborhoods in both cities are clustered in 10 groups, with neighborhoods in each group being similar to one another.

By using the maps and the tables generated in this notebook, the neighborhoods can be compared in great detail and can help you choose which neighborhood to move into.

## Step 1

We first import all the required libraries. 

In [3]:
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import json
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

## Step 2

Collect data on the neighborhoods of Toronto.

The BeautifulSoup package is used to extract the neighborhood data from the wikipedia page.

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

In [5]:
right_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

df['Postal Code']=df['Postal Code'].str.replace(r'\n', '')
df['Borough']=df['Borough'].str.replace(r'\n', '')
df['Neighborhood']=df['Neighborhood'].str.replace(r'\n', '')

In [6]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

coor = pd.read_csv('Geospatial_Coordinates.csv') # Extract Latitudes and Longitudes of neighborhoods
merge = df.merge(coor, how = 'inner', on = ['Postal Code'])
merge

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Step 3

Display the neighborhoods of Toronto on a geographical map.

In [7]:
address = 'Toronto, Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
latitute_toronto = latitude
longitude_toronto = longitude

toronto_data = merge

In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Step 4

Access Foursquare to get details of venues in each neighborhood of Toronto and use one-hot encoding to
 convert them into numerical values.

In [9]:
CLIENT_ID = '1Z1HNEJ3ALXU2X2RTZC0WJU5Z45RCEMDOKRNM3BJBFUDFU5F' 
CLIENT_SECRET = '0K3AQTBOACRVKXUASKNY035RU2TT2LTCIY4SS1BM4TXB2EZP' 
VERSION = '20180605' 

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
LIMIT=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

The dataframe 'toronoto_venues' contains details of the venues of all the neighborhoods.

In [12]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [13]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Step 5

Collect data on the neighborhoods of New York. Here we will only look at neighborhoods in the three most populous Boroughs of New York - Brooklyn, Queens and Manhattan.

In [15]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
    
neighborhoods_data = newyork_data['features']
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
manhattan_data = neighborhoods.loc[neighborhoods['Borough'].isin(['Manhattan','Brooklyn','Queens'])].reset_index(drop=True)
manhattan_data.shape


(191, 4)

In [33]:
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Brooklyn,Bay Ridge,40.625801,-74.030621
2,Brooklyn,Bensonhurst,40.611009,-73.995180
3,Brooklyn,Sunset Park,40.645103,-74.010316
4,Brooklyn,Greenpoint,40.730201,-73.954241


## Step 6

Display the neighborhoods of Brooklyn, Queens and Manhattan on a geographical maps.

In [16]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))
latitude_NY = latitude
longitude_NY=longitude

print(latitude_NY)

The geograpical coordinate of Manhattan are 40.7127281, -74.0060152.
40.7127281


In [17]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Step 7

Access Foursquare to get details of venues in each neighborhood and use one-hot encoding to
 convert them into numerical values.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho

In [22]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Step 8

Combine data from both cities to prepare them to be clustered based on indicators from FourSquare. In this way, similar neighborhoods from either city will be clustered into the same group, regardless of their geographical location.

In [24]:
both_onehot = toronto_onehot.append(manhattan_onehot)
both_onehot = both_onehot.fillna(0)
both_grouped = both_onehot.groupby('Neighborhood').mean().reset_index()
both_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Used Bookstore,Vape Store,Varenyky restaurant,Venezuelan Restaurant,Veterinarian,Video Store,Volleyball Court,Waterfront,Weight Loss Center,Whisky Bar
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,Arverne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,Astoria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,Astoria Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Woodhaven,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
280,Woodside,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
281,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
282,"York Mills, Silver Hills",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


## Step 9

Looking at the top ten venues in each neighborhood, we can already get an estimate of the similarity between different neighborhoods.

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = both_grouped['Neighborhood']

for ind in np.arange(both_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(both_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Clothing Store,Lounge,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pub,Sandwich Place,Pool,Skating Rink,Whisky Bar,Hospital,Hookah Bar
2,Arverne,Surf Spot,Metro Station,Sandwich Place,Beach,Café,Thai Restaurant,Pizza Place,Wine Shop,Bus Stop,Playground
3,Astoria,Bar,Middle Eastern Restaurant,Hookah Bar,Seafood Restaurant,Indian Restaurant,Greek Restaurant,Pizza Place,Mediterranean Restaurant,Café,Deli / Bodega
4,Astoria Heights,Supermarket,Business Service,Bakery,Bowling Alley,Burger Joint,Hostel,Bus Station,Pizza Place,Shopping Mall,Playground


## Step 10

Performing a clustering algorithm on the combined dataframe with information of neighbourhoods from both cities.
In principle, kclusters is a free parameter, but here we set it to k=50. The reason is clarified in the report.

In [26]:
kclusters = 50

both_grouped_clustering = both_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(both_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([23, 19, 37, 13, 13, 13, 19,  0, 14, 13], dtype=int32)

In [27]:
both_data = toronto_data.append(manhattan_data)

In [28]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
both_merged = both_data
both_merged = both_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
both_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,49.0,Food & Drink Shop,Construction & Landscaping,Park,Whisky Bar,Hotel,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum
1,M4A,North York,Victoria Village,43.725882,-79.315572,35.0,Intersection,Portuguese Restaurant,Coffee Shop,Hockey Arena,Pizza Place,Whisky Bar,Hostel,Harbor / Marina,Hardware Store,Health & Beauty Service
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Restaurant,Ice Cream Shop,Historic Site
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,37.0,Clothing Store,Accessories Store,Furniture / Home Store,Miscellaneous Shop,Event Space,Athletics & Sports,Women's Store,Boutique,Coffee Shop,Arts & Crafts Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Bar,Bank,Creperie,Burrito Place,Theater


## Step 11

Finally, we map out both cities to and use the same color code to identify similar neighborhoods in either city. 
For example, a "blue" neighborhood in Toronto is similar to a "blue" neighborhood in New York.

With this map, the user can be better informed about the location to choose when moving between the two cities.

In [29]:
both_merged = both_merged[both_merged['Cluster Labels'].notna()]
map_clusters = folium.Map(location=[latitute_toronto, longitude_toronto], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(both_merged['Latitude'], both_merged['Longitude'], both_merged['Neighborhood'], both_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude_NY, longitude_NY], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(both_merged['Latitude'], both_merged['Longitude'], both_merged['Neighborhood'], both_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Step 12

In-depth analysis of each cluster can be performed by looking at the top attributes of each neighbourhood in the cluster.
Here, a random cluster (label = 13) is chosen. 


In [104]:
both_merged.loc[both_merged['Cluster Labels'] == 13, both_merged.columns[[2] + list(range(5, both_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Glencairn,13.0,Asian Restaurant,Bakery,Japanese Restaurant,Pub,Italian Restaurant,Hostel,Hospital,Hookah Bar,Hockey Arena,Hobby Shop
31,"Dufferin, Dovercourt Village",13.0,Bakery,Pharmacy,Supermarket,Café,Bank,Park,Bar,Pet Store,Brewery,Grocery Store
37,"Little Portugal, Trinity",13.0,Bar,Asian Restaurant,Restaurant,Coffee Shop,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Yoga Studio,Dog Run
41,"The Danforth West, Riverdale",13.0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Liquor Store,Juice Bar,Bubble Tea Shop
43,"Brockton, Parkdale Village, Exhibition Place",13.0,Café,Bakery,Breakfast Spot,Coffee Shop,Convenience Store,Bar,Stadium,Gym,Nightclub,Burrito Place
...,...,...,...,...,...,...,...,...,...,...,...,...
173,Sunnyside Gardens,13.0,Bar,Grocery Store,Pizza Place,Pharmacy,Coffee Shop,Korean Restaurant,Mexican Restaurant,Food Truck,American Restaurant,Turkish Restaurant
176,Vinegar Hill,13.0,Food Truck,Coffee Shop,Café,Art Gallery,Whisky Bar,Performing Arts Venue,Entertainment Service,Ice Cream Shop,Latin American Restaurant,Wine Shop
179,Dumbo,13.0,Coffee Shop,Park,Scenic Lookout,Bakery,Yoga Studio,American Restaurant,Gym,Bookstore,Art Gallery,Pizza Place
182,Middle Village,13.0,Bakery,Playground,Sports Bar,Italian Restaurant,South American Restaurant,Sushi Restaurant,Dessert Shop,Sandwich Place,Bank,Diner
